# Precipitation Variability Across Timescales

This notebook demonstrates how to use the precipitation variability metrics driver and calc_ratio script to obtain the precipitation variability metric.

Our metric is based on the simulated-to-observed ratio of spectral power because the spectral power is substantially sensitive to the processing choices for power spectra analysis (e.g., window length, overlap length, and windowing function). By using the ratio, the metric is not greatly affected by the different processing choices, helping the robustness of analysis results.

This notebook should be run in an environment with python, jupyterlab, pcmdi metrics package, and cdat installed. It is expected that you have downloaded the sample data as demonstrated in [the download notebook](Demo_0_download_data.ipynb).  

The following cell reads in the choices you made during the download data step:

In [1]:
from user_choices import demo_data_directory, demo_output_directory

## Basic Use

### Help
Use the `--help` flag for assistance with the precip variability driver:

In [2]:
%%bash
variability_across_timescales_PS_driver.py --help

usage: variability_across_timescales_PS_driver.py [-h]
                                                  [--parameters PARAMETERS]
                                                  [--diags OTHER_PARAMETERS [OTHER_PARAMETERS ...]]
                                                  [--mip MIP] [--mod MOD]
                                                  [--var VAR] [--frq FRQ]
                                                  [--modpath MODPATH]
                                                  [--results_dir RESULTS_DIR]
                                                  [--case_id CASE_ID]
                                                  [--prd PRD [PRD ...]]
                                                  [--fac FAC]
                                                  [--nperseg NPERSEG]
                                                  [--noverlap NOVERLAP]
                                                  [--ref REF] [--cmec]
                                                  [--no_

### Parameter file
Settings can be specified in a parameter file or on the command line. The basic case demonstrated here uses a parameter file, which is printed below.  

Note that this driver should only be used to run **one** model or dataset at a time.  

The `mod` variable can either be set to a particular file name, as shown here, or to a model name (i.e. "GISS-E2-H").

In [3]:
# print parameter file
with open("basic_precip_variability_param.py") as f:
    print(f.read())

mip = "cmip5"
exp = "historical"
mod = "pr.day.GISS-E2-H.historical.r6i1p1.20000101-20051231.xml"
var = "pr"
frq = "day"
modpath = 'demo_data//CMIP5_demo_timeseries/historical/atmos/day/pr/'
results_dir = 'demo_output/precip_variability/GISS-E2-H/'
prd = [2000,2005]  # analysis period
fac = 86400  # factor to make unit of [mm/day]

# length of segment in power spectra (~10 years)
# shortened to 2 years for demo purposes
nperseg = 2 * 365
# length of overlap between segments in power spectra (~5 years)
# shortened to 1 year for demo purposes
noverlap = 1 * 365

# flag for cmec formatted JSON
cmec = False


### Model file

The precipitation variability metrics driver requires that model and observation file names follow this template:
`variable.frequency.model.experiment.realization.dates.extension`

Our sample model precipitation does not follow that format, so the next cell generates an XML file with a compliant name to use with the driver.

In [4]:
%%bash -s "$demo_data_directory"
cd $1/CMIP5_demo_timeseries/historical/atmos/day/pr/
cdscan -x  pr.day.GISS-E2-H.historical.r6i1p1.20000101-20051231.xml *.nc

Finding common directory ...
Common directory: 
Scanning files ...
pr_day_GISS-E2-H_historical_r6i1p1_20000101-20051231.nc
Setting reference time units to days since 2000-1-1
pr.day.GISS-E2-H.historical.r6i1p1.20000101-20051231.xml written


### Running the driver
The parameter file is passed to the driver using the `-p` flag, similar to other PMP metrics. The basic command is:  
`variability_across_timescales_PS_driver.py -p parameter_file_name.py`

The next cell uses the command line syntax to execute the driver as a subprocess.

In [5]:
%%bash
variability_across_timescales_PS_driver.py -p basic_precip_variability_param.py

demo_data//CMIP5_demo_timeseries/historical/atmos/day/pr/
pr.day.GISS-E2-H.historical.r6i1p1.20000101-20051231.xml
[2000, 2005]
730 365
demo_output/precip_variability/GISS-E2-H/
demo_output/precip_variability/GISS-E2-H/
demo_output/precip_variability/GISS-E2-H/
# of data: 1
['GISS-E2-H.historical']
GISS-E2-H.historical noleap
Complete regridding from (365, 90, 144) to (365, 90, 180)
2000 (365, 90, 180)
Complete regridding from (365, 90, 144) to (365, 90, 180)
2001 (730, 90, 180)
Complete regridding from (365, 90, 144) to (365, 90, 180)
2002 (1095, 90, 180)
Complete regridding from (365, 90, 144) to (365, 90, 180)
2003 (1460, 90, 180)
Complete regridding from (365, 90, 144) to (365, 90, 180)
2004 (1825, 90, 180)
Complete regridding from (365, 90, 144) to (365, 90, 180)
2005 (2190, 90, 180)
Complete calculating climatology and anomaly for calendar of noleap
Complete power spectra (segment:  730  nps: 5.0 )
Complete domain and frequency average of spectral power
Complete power spectra (se

INFO::2021-07-21 15:59::pcmdi_metrics:: Results saved to a json file: /Users/ordonez4/Documents/git/pcmdi_metrics/doc/jupyter/Demo/demo_output/precip_variability/GISS-E2-H/PS_pr.day_regrid.180x90_area.freq.mean_GISS-E2-H.historical.json


The following cell cleans up the data directory by removing the xml we made with the compatible name.

In [6]:
%%bash -s "$demo_data_directory"
rm $1/CMIP5_demo_timeseries/historical/atmos/day/pr/pr.day.GISS-E2-H.historical.r6i1p1.20000101-20051231.xml

### Results
Running the precipitation variability driver produces three output files, found in the demo output directory:  

Spatial pattern of spectral power (forced variability) (netCDF)   
Spatial pattern of spectral power (unforced variability) (netCDF)  
Average of spectral power (forced and unforced) (JSON)  

In [7]:
!ls {demo_output_directory + "/precip_variability/GISS-E2-H"}

PS_pr.day_regrid.180x90_GISS-E2-H.historical.nc
PS_pr.day_regrid.180x90_GISS-E2-H.historical_unforced.nc
PS_pr.day_regrid.180x90_area.freq.mean_GISS-E2-H.historical.json


The next cell displays the metrics from the JSON file.

In [8]:
import json
import os
output_path = os.path.join(demo_output_directory,"precip_variability/GISS-E2-H/PS_pr.day_regrid.180x90_area.freq.mean_GISS-E2-H.historical.json")
with open(output_path) as f:
    metric = json.load(f)["RESULTS"]
print(json.dumps(metric, indent=2))

{
  "GISS-E2-H.historical": {
    "forced": {
      "Land_30N50N": {
        "annual": 1.1549259632229734,
        "semi-annual": 0.36957929725177013
      },
      "Land_30S30N": {
        "annual": 6.86718228695229,
        "semi-annual": 1.1973130839359392
      },
      "Land_50S30S": {
        "annual": 0.7833102360471571,
        "semi-annual": 0.33415474921186816
      },
      "Land_50S50N": {
        "annual": 4.804355868733513,
        "semi-annual": 0.8992837441868183
      },
      "Ocean_30N50N": {
        "annual": 1.4471891957606835,
        "semi-annual": 0.37248020655592934
      },
      "Ocean_30S30N": {
        "annual": 4.5690821869910465,
        "semi-annual": 1.5047969231749088
      },
      "Ocean_50S30S": {
        "annual": 0.5919548020770604,
        "semi-annual": 0.19280887596650872
      },
      "Ocean_50S50N": {
        "annual": 3.310355604111882,
        "semi-annual": 1.0766772258953867
      },
      "Total_30N50N": {
        "annual": 1.3116137610

## Command line usage with Obs data

To calculate the precipitation variability spectral power ratio, we also need results for a reference dataset. This example shows how to call the `variability_across_timescales_PS_driver` using a combination of the parameter file and command line arguments with daily observational data. The command line arguments will overwrite values that are in the parameter file.  

The `modpath` and `results_dir` values are set first in a separate cell to easily combine the `demo_data_directory` and `demo_output_directory` variables with other strings. The new variables are then passed to the shell command in the second cell.

In [9]:
modpath = demo_data_directory + '/PCMDIobs2/atmos/day/pr/GPCP-IP/gn/v20200719/'
results_dir = demo_output_directory + '/precip_variability/GPCP-IP/'

In [10]:
%%bash -s "$modpath" "$results_dir"
variability_across_timescales_PS_driver.py -p basic_precip_variability_param.py \
--mip 'obs' \
--mod 'pr.day.GPCP-IP.BE.gn.v20200719.1998-1999.xml' \
--modpath $1 \
--results_dir $2 \
--prd 1998 1999

demo_data/PCMDIobs2/atmos/day/pr/GPCP-IP/gn/v20200719/
pr.day.GPCP-IP.BE.gn.v20200719.1998-1999.xml
[1998, 1999]
730 365
demo_output/precip_variability/GPCP-IP/
demo_output/precip_variability/GPCP-IP/
demo_output/precip_variability/GPCP-IP/
# of data: 1
['GPCP-IP']
GPCP-IP gregorian
Complete regridding from (365, 180, 360) to (365, 90, 180)
1998 (365, 90, 180)
Complete regridding from (365, 180, 360) to (365, 90, 180)
1999 (730, 90, 180)
Complete calculating climatology and anomaly for calendar of gregorian
Complete power spectra (segment:  730  nps: 1.0 )
Complete domain and frequency average of spectral power
Complete power spectra (segment:  730  nps: 1.0 )
Complete domain and frequency average of spectral power


/Users/ordonez4/miniconda3/envs/test_pcmdi/lib/python3.8/site-packages/pcmdi_metrics-v1.2.1_766_gf5058a32-py3.8.egg/pcmdi_metrics/precip_variability/lib/lib_variability_across_timescales.py:229: UserWarning: Warning: converting a masked element to nan.
INFO::2021-07-21 16:01::pcmdi_metrics:: Results saved to a json file: /Users/ordonez4/Documents/git/pcmdi_metrics/doc/jupyter/Demo/demo_output/precip_variability/GPCP-IP/PS_pr.day_regrid.180x90_area.freq.mean_GPCP-IP.json


## Precipitation Variability Metric

The precipitation variability metric can be generated after model and observational spectral averages are made. 

A script called "calc_ratio.py" is provided in the precip_variability codebase. This script can be called with three arguments to generate the ratio.  
`ref`: path to obs results JSON  
`modpath`: directory containing model results JSONS (not CMEC formatted JSONs)  
`results_dir`: directory for calc_ratio.py results

This script can be accessed via the PMP repo, which is how it is run here. It does not come with the PMP conda installation.

In [11]:
%%bash -s "$demo_output_directory"
python ../../../pcmdi_metrics/precip_variability/scripts_pcmdi/calc_ratio.py \
--ref $1/precip_variability/GPCP-IP/PS_pr.day_regrid.180x90_area.freq.mean_GPCP-IP.json \
--modpath $1/precip_variability/GISS-E2-H/ \
--results_dir $1/precip_variability/ratio/

reference:  demo_output/precip_variability/GPCP-IP/PS_pr.day_regrid.180x90_area.freq.mean_GPCP-IP.json
modpath:  demo_output/precip_variability/GISS-E2-H/
outdir:  demo_output/precip_variability/ratio/
['demo_output/precip_variability/GISS-E2-H/PS_pr.day_regrid.180x90_area.freq.mean_GISS-E2-H.historical.json']
Complete  GISS-E2-H.historical
Complete all


This outputs one JSON file in the `results_dir` folder. The results in this file are shown below.

In [12]:
output_path = os.path.join(demo_output_directory,"precip_variability/ratio/PS_pr.day_regrid.180x90_area.freq.mean_GISS-E2-H.historical.json")
with open(output_path) as f:
    metric = json.load(f)["RESULTS"]
print(json.dumps(metric, indent=2))

{
  "GISS-E2-H.historical": {
    "forced": {
      "Land_30N50N": {
        "annual": 1.1500194464027775,
        "semi-annual": 1.553216653910303
      },
      "Land_30S30N": {
        "annual": 1.295541384665899,
        "semi-annual": 1.488996362077941
      },
      "Land_50S30S": {
        "annual": 1.5143757307185535,
        "semi-annual": 2.4937185698342192
      },
      "Land_50S50N": {
        "annual": 1.1618572684631605,
        "semi-annual": 1.3870817424737694
      },
      "Ocean_30N50N": {
        "annual": 0.9198764990934699,
        "semi-annual": 0.7395557740837525
      },
      "Ocean_30S30N": {
        "annual": 1.5119666276458792,
        "semi-annual": 1.886140600092445
      },
      "Ocean_50S30S": {
        "annual": 1.46247816502199,
        "semi-annual": 1.0417004965116758
      },
      "Ocean_50S50N": {
        "annual": 1.4584271041573642,
        "semi-annual": 1.7161187315455917
      },
      "Total_30N50N": {
        "annual": 0.9806701388850977